In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O tiny_shakespear.txt

--2023-11-29 00:02:56--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.


HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘tiny_shakespear.txt’

tiny_shakespear.txt 100%[===================>]   1.06M  3.53MB/s    in 0.3s    

2023-11-29 00:02:57 (3.53 MB/s) - ‘tiny_shakespear.txt’ saved [1115394/1115394]



In [2]:
with open("tiny_shakespear.txt", 'r', encoding='utf-8') as handle:
    text = handle.read()

In [3]:
print(f"Length of the dataset: {len(text)}")

Length of the dataset: 1115394


In [4]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [5]:
characters = sorted(
    list(
        set(
            text
        )
    )
)
vocabulary_size = len(characters)
print(f"All available characters: {''.join(characters)}")
print(f"Their count: {vocabulary_size}")

All available characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Their count: 65


In [6]:
string_to_int = {char: index for index, char in enumerate(characters)}
int_to_string = {index: char for index, char in enumerate(characters)}

encode = lambda string: [string_to_int[character] for character in string]
decode = lambda list_of_integers: ''.join([int_to_string[integer] for integer in list_of_integers])

print(f"{encode('hi mom!')=}")
print(f"{decode(encode('hi mom!'))=}")

encode('hi mom!')=[46, 47, 1, 51, 53, 51, 2]
decode(encode('hi mom!'))='hi mom!'


In [7]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(f"{data.shape=}, {data.dtype=}")
print(f"{data[:200]=}")

/home/kali/rep/me/side-projects/python/karpathy-gpt/.venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


data.shape=torch.Size([1115394]), data.dtype=torch.int64
data[:200]=tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59])


In [8]:
n = int (0.9 * len(data))

train_data = data[:n]
validation_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1: block_size + 1]

for token in range(block_size):
    context = x[: token + 1]
    target = y[token]
    print(f"when input is {context} the target is: {target}")

when input is tensor([18]) the target is: 47
when input is tensor([18, 47]) the target is: 56
when input is tensor([18, 47, 56]) the target is: 57
when input is tensor([18, 47, 56, 57]) the target is: 58
when input is tensor([18, 47, 56, 57, 58]) the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [11]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else validation_data
    
    ix = torch.randint(
        len(data) - block_size, 
        (batch_size,)
    )
    x = torch.stack([
        data[i: i + block_size]
        for i in ix
    ])
    y = torch.stack([
        data[i + 1: i + block_size + 1]
        for i in ix
    ])
    return x, y
    
xb, yb = get_batch('train')
print(f'INPUTS {xb.shape=}\n {xb=}\n')
print(f'TARGETS {yb.shape=}\n {yb=}')

for batch in range(batch_size):
    for token in range(block_size):
        context = xb[batch, : token + 1]
        target = yb[batch, token]
        print(f"when input is {context.tolist()} the target: {target}")

INPUTS xb.shape=torch.Size([4, 8])
 xb=tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

TARGETS yb.shape=torch.Size([4, 8])
 yb=tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
w

In [23]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocabulary_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocabulary_size, vocabulary_size)
                                                 
    def forward(self, idx, targets = None):    
        logits = self.token_embedding_table(idx) 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)            
            loss = F.cross_entropy(logits, targets)       
            
        return logits, loss
        
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T + 1)
        return idx
        
model = BigramLanguageModel(vocabulary_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

print(decode(
    model.generate(
        torch.zeros((1, 1), dtype=torch.long),
        max_new_tokens=100
    )[0].tolist()
))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [26]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [31]:
batch_size = 32

for steps in range(10000):
    xb, yb = get_batch('train')
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.6319539546966553


In [33]:
print(decode(
    model.generate(
        torch.zeros((1, 1), dtype=torch.long),
        max_new_tokens=200
    )[0].tolist()
))


Te m inoy fot athad tatanter hindainishid'?
We
O: heandio ongrday baze s t id olorayourkees be! bet hous thir

Win:
MESl'
Whro y pomal blen?

AR:
A y'l beee inisourders s be,
To pow theathiehe; e sclk
